In [2]:
import praw

# Initialize a Reddit instance
reddit = praw.Reddit(
    client_id="R4aTlqzdkwL8HlP0kbqI_w",
    client_secret="_-xw-M0CWgf3xDc7XMJU3RdCWB9WIQ",
    user_agent="Hazel/1.0 by SeaLimit6194"
)

# Example: Get the top posts from a subreddit
subreddit = reddit.subreddit("python")
for submission in subreddit.top(limit=5):
    print(submission.title)


Lad wrote a Python script to download Alexa voice recordings, he didn't expect this email.
This post has:
I redesign the Python logo to make it more modern
Automate the boring stuff with python - tinder
I'm excited to share my first published book, Introduction to Python Programming for Business and Social Science Applications -- specifically geared towards students not specifically in computer science


In [ ]:
import praw
import json
import os
import sys

# Initialize Reddit instance with credentials
reddit = praw.Reddit(
    client_id="R4aTlqzdkwL8HlP0kbqI_w",            # Replace with your client_id
    client_secret="_-xw-M0CWgf3xDc7XMJU3RdCWB9WIQ",    # Replace with your client_secret
    user_agent="Hazel/1.0 by SeaLimit6194"
)

# Subreddits to scrape
subreddits = ["politics", "worldnews", "marketing", "all"]

# Target file size (in bytes)
target_file_size = 500 * 1024 * 1024  # 500 MB

# File to save the data
output_file = "reddit_data.json"

# Check if output file already exists and remove it to start fresh
if os.path.exists(output_file):
    os.remove(output_file)

# Initialize the data list
data = []

# Function to calculate the file size
def get_file_size(file_path):
    if os.path.exists(file_path):
        return os.path.getsize(file_path)
    return 0

# Function to fetch posts from a subreddit
def fetch_posts(subreddit_name, limit=1000):
    subreddit = reddit.subreddit(subreddit_name)
    posts = []

    # Fetch top and hot posts
    for submission in subreddit.hot(limit=limit):
        posts.append(submission)
    for submission in subreddit.top(limit=limit):
        posts.append(submission)

    return posts

try:
    # Fetch posts from each subreddit and accumulate data
    for subreddit_name in subreddits:
        print(f"Fetching data from r/{subreddit_name}...")
        posts = fetch_posts(subreddit_name)

        # Process each post and save necessary data
        for submission in posts:
            post_data = {
                "title": submission.title,
                "score": submission.score,
                "author": str(submission.author),
                "created_utc": submission.created_utc,
                "subreddit": submission.subreddit.display_name,
                "selftext": submission.selftext,
                "url": submission.url
            }
            data.append(post_data)

            # Save data to JSON file periodically to check file size
            with open(output_file, "w") as f:
                json.dump(data, f, indent=4)

            # Check the file size after each save
            if get_file_size(output_file) >= target_file_size:
                print("Target file size reached (500MB). Stopping...")
                sys.exit(0)

except Exception as e:
    print(f"An error occurred: {e}")

print("Data collection complete.")


In [5]:
print(f"Saved {len(data)} posts to {output_file}, current size: {get_file_size(output_file) / (1024 * 1024):.2f} MB")


Saved 7242 posts to reddit_data.json, current size: 5.69 MB


In [9]:
import praw
import json
import time
import os

# Initialize Reddit instance with credentials
reddit = praw.Reddit(
    client_id="R4aTlqzdkwL8HlP0kbqI_w",            # Replace with your client_id
    client_secret="_-xw-M0CWgf3xDc7XMJU3RdCWB9WIQ",    # Replace with your client_secret
    user_agent="Hazel/1.0 by SeaLimit6194"
)
# Expanded list of popular, marketing, and politics-related subreddits
subreddits = [
    "politics", "worldnews", "news", "Ask_Politics", "PoliticalDiscussion",
    "ukpolitics", "canadapolitics", "europe", "australianpolitics", 
    "China", "IndianPolitics", "MiddleEastNews", "uspolitics", 
    "AsiaNews", "conspiracy", "geopolitics", "Libertarian", "democrats", "republican",
    "marketing", "DigitalMarketing", "SEO", "content_marketing", "advertising",
    "AskMarketing", "Entrepreneur", "smallbusiness", "business", "technology", "science"
]

# Number of posts to collect per subreddit per category
posts_per_subreddit = 200

# Categories of posts to collect (hot, new, rising, controversial)
categories = ["hot", "new", "rising", "controversial"]

# Output file for saving data
output_file = "politics_and_marketing_data.json"

# Load existing data to avoid duplicates
if os.path.exists(output_file):
    with open(output_file, "r") as f:
        data = json.load(f)
    # Track existing post IDs to prevent duplicates
    existing_ids = {post['id'] for post in data}
else:
    data = []
    existing_ids = set()

# Function to fetch posts from a subreddit in different categories
def fetch_subreddit_posts(subreddit_name, category):
    subreddit = reddit.subreddit(subreddit_name)
    posts = []
    
    # Select the appropriate category
    if category == "hot":
        submissions = subreddit.hot(limit=posts_per_subreddit)
    elif category == "new":
        submissions = subreddit.new(limit=posts_per_subreddit)
    elif category == "rising":
        submissions = subreddit.rising(limit=posts_per_subreddit)
    elif category == "controversial":
        submissions = subreddit.controversial(limit=posts_per_subreddit)

    # Process each submission
    for submission in submissions:
        if submission.id not in existing_ids:  # Avoid duplicates by ID
            post_data = {
                "id": submission.id,
                "title": submission.title,
                "score": submission.score,
                "author": str(submission.author),
                "created_utc": submission.created_utc,
                "subreddit": submission.subreddit.display_name,
                "selftext": submission.selftext,
                "url": submission.url,
                "category": category  # Track the category (hot, new, rising, controversial)
            }
            posts.append(post_data)
            existing_ids.add(submission.id)  # Add ID to the set to avoid duplicates

    return posts

# Collect data for each subreddit and category
for subreddit_name in subreddits:
    for category in categories:
        print(f"Fetching {category} posts from r/{subreddit_name}...")
        data.extend(fetch_subreddit_posts(subreddit_name, category))
        time.sleep(2)  # Sleep to avoid rate limiting

# Save the collected data to JSON
with open(output_file, "w") as f:
    json.dump(data, f, indent=4)

print("Data collection complete. Saved to politics_and_marketing_data.json")


Fetching hot posts from r/politics...
Fetching new posts from r/politics...
Fetching rising posts from r/politics...
Fetching controversial posts from r/politics...
Fetching hot posts from r/worldnews...
Fetching new posts from r/worldnews...
Fetching rising posts from r/worldnews...
Fetching controversial posts from r/worldnews...
Fetching hot posts from r/news...
Fetching new posts from r/news...
Fetching rising posts from r/news...
Fetching controversial posts from r/news...
Fetching hot posts from r/Ask_Politics...
Fetching new posts from r/Ask_Politics...
Fetching rising posts from r/Ask_Politics...
Fetching controversial posts from r/Ask_Politics...
Fetching hot posts from r/PoliticalDiscussion...
Fetching new posts from r/PoliticalDiscussion...
Fetching rising posts from r/PoliticalDiscussion...
Fetching controversial posts from r/PoliticalDiscussion...
Fetching hot posts from r/ukpolitics...
Fetching new posts from r/ukpolitics...
Fetching rising posts from r/ukpolitics...
Fetc

Forbidden: received 403 HTTP response

In [10]:
data = fetch_subreddit_posts("funny", "hot")
print(data)


[{'id': '14jmh7e', 'title': 'Forging A Return to Productive Conversation: An Open Letter to Reddit', 'score': 1193, 'author': 'Funny_Sentinel', 'created_utc': 1687798111.0, 'subreddit': 'funny', 'selftext': 'To All Whom It May Concern:\n\nFor fifteen years, /r/Funny has been one of Reddit’s most-popular communities. That time hasn’t been without its difficulties, but for the most part, we’ve all gotten along (with each other and with administrators). Members of our team fondly remember Moderator Roadshows, visits to Reddit’s headquarters, Reddit Secret Santa, April Fools’ Day events, regional meetups, and many more uplifting moments. We’ve watched this platform grow by leaps and bounds, and although we haven’t been completely happy about every change that we’ve witnessed, we’ve always done our best to work with Reddit at finding ways to adapt, compromise, and move forward. \n\nThis process has occasionally been preceded by some exceptionally public debate, however.\n\nOn June 12th, 202

In [12]:
data = fetch_subreddit_posts("funny", "hot")
print(data)


[]


In [13]:
import praw
import os

# Initialize Reddit instance
reddit = praw.Reddit(
    client_id="R4aTlqzdkwL8HlP0kbqI_w",            # Replace with your client_id
    client_secret="_-xw-M0CWgf3xDc7XMJU3RdCWB9WIQ",    # Replace with your client_secret
    user_agent="Hazel/1.0 by SeaLimit6194"
)

# Test with a single post from a public subreddit
def test_fetch():
    try:
        subreddit = reddit.subreddit("AskReddit")
        submissions = subreddit.hot(limit=5)
        for submission in submissions:
            print(f"Title: {submission.title}")
    except Exception as e:
        print("Error:", e)

# Run the test
test_fetch()


Title: Halloween Megathread 2024
Title: Sexy Halloween X
Title: People who have successfully quit smoking, how did you do it?
Title: Which discontinued fast food item do you want to come back?
Title: What illusions did you have about sex due to watching porn?


In [15]:
import praw
import json
import time
import os

# Initialize Reddit instance with credentials from environment variables
reddit = praw.Reddit(
    client_id="R4aTlqzdkwL8HlP0kbqI_w",            # Replace with your client_id
    client_secret="_-xw-M0CWgf3xDc7XMJU3RdCWB9WIQ",    # Replace with your client_secret
    user_agent="Hazel/1.0 by SeaLimit6194"
)

# List of subreddits to scrape
subreddits = ["politics", "worldnews", "news", "AskReddit", "technology", "funny"]

# Categories to retrieve posts from
categories = ["hot", "new", "rising"]

# Set a manageable limit per category
posts_per_subreddit = 100
output_file = "reddit_data.json"  # Output file name

# Load existing data to avoid duplicates if file already exists
if os.path.exists(output_file):
    with open(output_file, "r") as f:
        data = json.load(f)
    # Track existing post IDs to prevent duplicates, ensuring only entries with 'id' key are used
    existing_ids = {post['id'] for post in data if 'id' in post}
else:
    data = []
    existing_ids = set()

# Fetch function with deduplication and delay
def fetch_subreddit_posts(subreddit_name, category):
    subreddit = reddit.subreddit(subreddit_name)
    posts = []

    try:
        if category == "hot":
            submissions = subreddit.hot(limit=posts_per_subreddit)
        elif category == "new":
            submissions = subreddit.new(limit=posts_per_subreddit)
        elif category == "rising":
            submissions = subreddit.rising(limit=posts_per_subreddit)

        for submission in submissions:
            if submission.id not in existing_ids:  # Avoid duplicates by ID
                post_data = {
                    "id": submission.id,
                    "title": submission.title,
                    "score": submission.score,
                    "author": str(submission.author),
                    "created_utc": submission.created_utc,
                    "subreddit": submission.subreddit.display_name,
                    "selftext": submission.selftext,
                    "url": submission.url,
                    "category": category
                }
                posts.append(post_data)
                existing_ids.add(submission.id)  # Add ID to the set to avoid duplicates
    except Exception as e:
        print(f"Error fetching {category} posts from r/{subreddit_name}: {e}")

    return posts

# Collect and save data
for subreddit_name in subreddits:
    for category in categories:
        print(f"Fetching {category} posts from r/{subreddit_name}...")
        data.extend(fetch_subreddit_posts(subreddit_name, category))
        time.sleep(5)  # Delay of 5 seconds between requests to manage rate limiting

# Save the collected data to JSON
with open(output_file, "w") as f:
    json.dump(data, f, indent=4)

print(f"Data collection complete. Saved to {output_file}")


Fetching hot posts from r/politics...
Fetching new posts from r/politics...
Fetching rising posts from r/politics...
Fetching hot posts from r/worldnews...
Fetching new posts from r/worldnews...
Fetching rising posts from r/worldnews...
Fetching hot posts from r/news...
Fetching new posts from r/news...
Fetching rising posts from r/news...
Fetching hot posts from r/AskReddit...
Fetching new posts from r/AskReddit...
Fetching rising posts from r/AskReddit...
Fetching hot posts from r/technology...
Fetching new posts from r/technology...
Fetching rising posts from r/technology...
Fetching hot posts from r/funny...
Fetching new posts from r/funny...
Fetching rising posts from r/funny...
Data collection complete. Saved to reddit_data.json


In [ ]:
import praw
import json
import time
import os

# Initialize Reddit instance with credentials
reddit = praw.Reddit(
    client_id="R4aTlqzdkwL8HlP0kbqI_w",            # Replace with your client_id
    client_secret="_-xw-M0CWgf3xDc7XMJU3RdCWB9WIQ", # Replace with your client_secret
    user_agent="Hazel/1.0 by SeaLimit6194"
)

# List of subreddits
subreddits = [
    "politics", "worldnews", "news", "AskReddit", "technology", "science",
    "funny", "gaming", "movies", "television", "books", "music",
    "health", "nutrition", "Fitness", "medicine", "space",
    "sports", "soccer", "nba", "nfl", "baseball", "hockey",
    "finance", "investing", "cryptocurrency", "personalfinance",
    "travel", "photography", "Art", "history", "AskHistorians",
    "explainlikeimfive", "lifeprotips", "DIY", "cars", "food",
    "marketing", "DigitalMarketing", "SEO", "content_marketing", "advertising",
    "Entrepreneur", "smallbusiness", "business", "SocialMedia", "startups"
]

# Categories to retrieve posts from
categories = ["hot", "new", "rising", "controversial", "top"]

# Settings
posts_per_subreddit = 500
output_file = "reddit_data.json"
save_interval = 100  # Save data every 100 posts to avoid data loss

# Load existing data and IDs to avoid duplicates
if os.path.exists(output_file):
    with open(output_file, "r") as f:
        data = json.load(f)
    existing_ids = {post['id'] for post in data if 'id' in post}
else:
    data = []
    existing_ids = set()

# Fetch function with deduplication
def fetch_subreddit_posts(subreddit_name, category):
    subreddit = reddit.subreddit(subreddit_name)
    posts = []
    
    try:
        submissions = getattr(subreddit, category)(limit=posts_per_subreddit)
        
        for submission in submissions:
            if submission.id not in existing_ids:
                post_data = {
                    "id": submission.id,
                    "title": submission.title,
                    "score": submission.score,
                    "author": str(submission.author),
                    "created_utc": submission.created_utc,
                    "subreddit": subreddit.display_name,
                    "selftext": submission.selftext,
                    "url": submission.url,
                    "category": category
                }
                posts.append(post_data)
                existing_ids.add(submission.id)
    except Exception as e:
        print(f"Error fetching {category} posts from r/{subreddit_name}: {e}")
    return posts

# Data collection with periodic saving and size limit
post_count = 0
for _ in range(20):
    for subreddit_name in subreddits:
        for category in categories:
            print(f"Fetching {category} posts from r/{subreddit_name}...")
            new_posts = fetch_subreddit_posts(subreddit_name, category)
            data.extend(new_posts)
            post_count += len(new_posts)
            time.sleep(5)

            # Save periodically to avoid data loss
            if post_count >= save_interval:
                with open(output_file, "w") as f:
                    json.dump(data, f, indent=4)
                    f.flush()
                print(f"Saved {post_count} posts to {output_file}")
                post_count = 0  # Reset counter after save

            # Check file size limit
            if os.path.getsize(output_file) >= 500 * 1024 * 1024:  # 500 MB
                print("Target file size of 500 MB reached. Stopping...")
                break
        else:
            continue
        break

print(f"Data collection complete. Saved to {output_file}")


In [ ]:
import praw
import json
import time
import os
import threading
import sys

# Initialize Reddit instance with credentials
reddit = praw.Reddit(
    client_id="R4aTlqzdkwL8HlP0kbqI_w",            # Replace with your client_id
    client_secret="_-xw-M0CWgf3xDc7XMJU3RdCWB9WIQ", # Replace with your client_secret
    user_agent="Hazel/1.0 by SeaLimit6194"
)

# List of subreddits
subreddits = [
    "politics", "worldnews", "news", "AskReddit", "technology", "science",
    "funny", "gaming", "movies", "television", "books", "music",
    "health", "nutrition", "Fitness", "medicine", "space",
    "sports", "soccer", "nba", "nfl", "baseball", "hockey",
    "finance", "investing", "cryptocurrency", "personalfinance",
    "travel", "photography", "Art", "history", "AskHistorians",
    "explainlikeimfive", "lifeprotips", "DIY", "cars", "food",
    "marketing", "DigitalMarketing", "SEO", "content_marketing", "advertising",
    "Entrepreneur", "smallbusiness", "business", "SocialMedia", "startups"
]

# Categories to retrieve posts from
categories = ["hot", "new", "rising", "controversial", "top"]

# Settings
posts_per_subreddit = 500
output_file = "reddit_data.json"
save_interval = 50  # Save data every 50 posts to avoid data loss

# Flag to control graceful shutdown
is_running = True

# Load existing data and IDs to avoid duplicates
if os.path.exists(output_file):
    with open(output_file, "r") as f:
        data = json.load(f)
    existing_ids = {post['id'] for post in data if 'id' in post}
else:
    data = []
    existing_ids = set()

# Fetch function with deduplication
def fetch_subreddit_posts(subreddit_name, category):
    subreddit = reddit.subreddit(subreddit_name)
    posts = []
    
    try:
        submissions = getattr(subreddit, category)(limit=posts_per_subreddit)
        
        for submission in submissions:
            if submission.id not in existing_ids:
                post_data = {
                    "id": submission.id,
                    "title": submission.title,
                    "score": submission.score,
                    "author": str(submission.author),
                    "created_utc": submission.created_utc,
                    "subreddit": subreddit.display_name,
                    "selftext": submission.selftext,
                    "url": submission.url,
                    "category": category
                }
                posts.append(post_data)
                existing_ids.add(submission.id)
    except Exception as e:
        print(f"Error fetching {category} posts from r/{subreddit_name}: {e}")
    return posts

# Thread function to listen for "Ctrl+P" input for graceful shutdown
def listen_for_exit():
    global is_running
    print("Press 'Ctrl+P' to stop scraping and save progress.")
    while is_running:
        if input() == '\x10':  # '\x10' is ASCII code for Ctrl+P
            print("Ctrl+P detected. Initiating graceful shutdown...")
            is_running = False

# Start the listener thread for Ctrl+P input
threading.Thread(target=listen_for_exit, daemon=True).start()

# Data collection with periodic saving and size limit
post_count = 0
try:
    for _ in range(50):  # Increased loop count to accumulate more data
        if not is_running:
            break
        for subreddit_name in subreddits:
            if not is_running:
                break
            for category in categories:
                if not is_running:
                    break
                print(f"Fetching {category} posts from r/{subreddit_name}...")
                new_posts = fetch_subreddit_posts(subreddit_name, category)
                data.extend(new_posts)
                post_count += len(new_posts)
                time.sleep(2)

                # Save periodically to avoid data loss
                if post_count >= save_interval:
                    with open(output_file, "w") as f:
                        json.dump(data, f, indent=4)
                        f.flush()
                    print(f"Saved {post_count} posts to {output_file}")
                    post_count = 0  # Reset counter after save

                # Check file size limit
                if os.path.getsize(output_file) >= 500 * 1024 * 1024:  # 500 MB
                    print("Target file size of 500 MB reached. Stopping...")
                    is_running = False
                    break
except Exception as e:
    print(f"An error occurred: {e}")

# Final save before exit
if not is_running or post_count > 0:
    with open(output_file, "w") as f:
        json.dump(data, f, indent=4)
    print("Final save completed before exit.")

print(f"Data collection complete. Saved to {output_file}")


Press 'Ctrl+P' to stop scraping and save progress.Fetching hot posts from r/politics...

Fetching new posts from r/politics...
Fetching rising posts from r/politics...
Fetching controversial posts from r/politics...
Fetching top posts from r/politics...
Fetching hot posts from r/worldnews...
Fetching new posts from r/worldnews...
Fetching rising posts from r/worldnews...
Fetching controversial posts from r/worldnews...
Fetching top posts from r/worldnews...
Fetching hot posts from r/news...
Fetching new posts from r/news...
Fetching rising posts from r/news...
Fetching controversial posts from r/news...
Fetching top posts from r/news...
Fetching hot posts from r/AskReddit...
Saved 67 posts to reddit_data.json
Fetching new posts from r/AskReddit...
Fetching rising posts from r/AskReddit...
Fetching controversial posts from r/AskReddit...
Fetching top posts from r/AskReddit...
Fetching hot posts from r/technology...
Fetching new posts from r/technology...
Fetching rising posts from r/tec